In [2]:
import vertexai # to interact with googles code chatbot ai
from vertexai.preview.language_models import CodeChatModel, ChatModel, InputOutputTextPair, ChatMessage,CodeGenerationModel, TextGenerationModel
import google
import openai


import syfco # to convert between different form
import bosy

import benchmark
from benchmark import Benchmark, build_prompt

import verify # used to verify verilog solutions
import prompting # prompting helper class
import json
import os

# both for handling async stuff
import asyncio

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

import csv # to write the benchmark results to a csv file
from utils import *

In [4]:
def generate_module_signature(bm: Benchmark, params=None, module_name="fsm"):
    if params == None:
        params = bm.generate_params
    spec = read_file(bm.specification)
    inputs = syfco.inputs(spec, overwrite_params=params)
    outputs = syfco.outputs(spec, overwrite_params=params)
    return f"""module {module_name} ({
    ", ".join(
      ["input " + inp for inp in inputs] + ["output reg " + out for out in outputs]
    )
  })"""


Used packages:
google-cloud-aiplatform
docker

In [3]:
vertexai.init(project="rg-finkbeiner-30141001", location="us-central1")
chat_model = ChatModel.from_pretrained("chat-bison")
code_model = CodeChatModel.from_pretrained("codechat-bison")
parameters = {
    "temperature": 0.5,
    "max_output_tokens": 1024
}
openai.api_key = os.getenv("OPENAI_KEY")

In [5]:
benchmarks = [
    Benchmark(bm, "../../verilog/") for bm in json.loads(read_file("benchmarks.json"))
]
benchmarks_auto = [
    Benchmark(bm, "") for bm in json.loads(read_file("benchmarks_auto.json"))
]

In [10]:
from prompting import interpolate_string
import time

class ExamplesPrompt(prompting.DefaultPromptTemplate):
  def add_example(self, replacements: dict = {}):
    pair = InputOutputTextPair(replacements["SPEC"], "```verilog\n" + replacements["IMPL"] + "\n```")
    self.examples.append(pair)
  def build_prompt(self, replacements: dict = {}):
    return interpolate_string(
      self._start + "\n" + self._question,
      replacements,
    ), self.examples
class ExpertPrompt(prompting.DefaultPromptTemplate):
  _start = "Simulate a brilliant computer scientist which is an expert in writing Verilog code such that it satisfies a specification in LTL. At first the expert will be shown some automatically generated examples fro smaller parameter values. Afterwards, he will be asked to write a solution for a bigger specification. It is of upmost importance that the code follows the specification. The prior examples can be used as a basis to start from."
  _example = "Here is a correct example for %PARAMS%. It satisfies the LTL specification %SPEC%:\n%IMPL%"
  _question = "Please write a Verilog module fulfilling the following expectations. Make sure the code is fully synthesizable. Only output the verilog module and nothing else. LTL Specification:\n%SPEC%"

class OpenAIPrompt(prompting.DefaultPromptTemplate):
  def add_example(self, replacements: dict = {}):
    pair = (replacements["SPEC"], "```verilog\n" + replacements["IMPL"] + "\n```")
    self.examples.append(pair)
  def build_prompt(self, replacements: dict = {}):
    messages = [
      {"role": "system", "content": self._start}
    ]
    for spec, code in self.examples:
      messages += [
        {"role": "user", "content": interpolate_string(self._question, replacements={"SPEC": spec})},
        {"role": "assistant", "content": code}
      ]
    messages.append({
      "role": "user",
      "content": interpolate_string(self._question, replacements)
    })
    return messages


In [13]:

def improve_iteratively(response, bm):
  code = extract_normalized_verilog_code(response, bm.name)
  res = "NO_CODE" if code == None else verify.verify_code(
    bm.specification, code, overwrite_params=bm.generate_params
    ).name
  match res:
    case "NO_CODE":
      prompt = "This doesn't seem like a complete verilog module. Could you please give me the entire module?"
    case "ERROR_CONVERT_TO_VERILOG":
      prompt = "This code isn't syntactically valid verilog code. Could you please fix the syntax errors?"
    case "ERROR_COMBINE_AIGER":
      prompt = f"Could you please make sure to use the module signature `{generate_module_signature(bm)}` in your code. Please also try to think about your code again using the new signature. Make sure it matches the specification!"
    case "FALSE_RESULT":
      prompt = "The code doesn't satisfy the specification. Please think extensively about how you need to change your code to satisfy the specification. If necessary, rewrite the module from ground up"
    case "SUCCESS":
      return (code, res)
    case _:
      return (None, res)
  return (prompt, res)


def run_single_iterative(bm, type):
  # bm.build_prompt might do some heavy work like synthesizing examples with bosy
  prompt, examples = build_prompt(bm, params=bm.generate_params, template=ExamplesPrompt, mode=type)
  chat = chat_model.start_chat(examples=examples)
  response = chat.send_message(prompt, **parameters)
  for i in range(0, 3):
    (prompt, res) = improve_iteratively(response.text, bm)
    if res == "SUCCESS" or res == None:
      return res
    response = chat.send_message(prompt, **parameters)
  return res

def run_single_explicit_examples(bm, type):
    print("Starting benchmark " + bm.name + "/" + type)
    # bm.build_prompt might do some heavy work like synthesizing examples with bosy
    prompt, examples = build_prompt(bm, params=bm.generate_params, template=ExamplesPrompt, mode=type)
    print("Finished generating prompt for " + bm.name + "/" + type)
    chat = chat_model.start_chat(examples=examples)
    response = chat.send_message(prompt, **parameters)
    code = extract_normalized_verilog_code(response.text, bm.name)
    # print(code if code else "NO_CODE::\n" + response.text)
    if code == None:
      return "NO_CODE"
    else:
      res = verify.verify_code(bm.specification, code, overwrite_params=bm.generate_params)
      return res.name

def run_single_best_k(k):
  def run_single (bm, type):
    # bm.build_prompt might do some heavy work like synthesizing examples with bosy
    prompt, examples = build_prompt(bm, params=bm.generate_params, template=ExamplesPrompt, mode=type)
    best = "NO_CODE"
    for i in range(0, k):
      chat = chat_model.start_chat(examples=examples)
      response = chat.send_message(prompt, **parameters)
      code = extract_normalized_verilog_code(response.text, bm.name)
      if code == None:
        continue
      res = verify.verify_code(bm.specification, code, overwrite_params=bm.generate_params)
      if res ==  verify.ReturnCode.SUCCESS:
        return res.name
      elif res == verify.ReturnCode.FALSE_RESULT:
        best = res.name
      elif best != "FALSE_RESULT":
        best = res.name
    return best
  return run_single

def run_single_default(bm, type, template=prompting.DefaultPromptTemplate):
    print("Starting benchmark " + bm.name + "/" + type)
    # bm.build_prompt might do some heavy work like synthesizing examples with bosy
    prompt = build_prompt(bm, params=bm.generate_params, template=template, mode=type)
    print(prompt)
    print("Finished generating prompt for " + bm.name + "/" + type)
    chat = chat_model.start_chat()
    response = chat.send_message(prompt, **parameters)
    code = extract_normalized_verilog_code(response.text, bm.name)
    print(code if code else "NO_CODE::\n" + response.text)
    if code == None:
      return "NO_CODE"
    else:
      res = verify.verify_code(bm.specification, code, overwrite_params=bm.generate_params)
      return res.name

def run_single_template(template=prompting.DefaultPromptTemplate):
  def run_single(bm, type):
    prompt = build_prompt(bm, params=bm.generate_params, template=template, mode=type)
    chat = chat_model.start_chat()
    response = chat.send_message(prompt, **parameters)
    code = extract_normalized_verilog_code(response.text, bm.name)
    if code == None:
      print(response.text)
      return "NO_CODE"
    else:
      res = verify.verify_code(bm.specification, code, overwrite_params=bm.generate_params)
      return res.name
  return run_single

def run_single_codechat(bm, type, template=prompting.DefaultPromptTemplate):
  print("Starting benchmark " + bm.name + "/" + type)
  # bm.build_prompt might do some heavy work like synthesizing examples with bosy
  prompt = build_prompt(bm, params=bm.generate_params, template=template, mode=type)
  chat = code_model.start_chat()
  response = chat.send_message(prompt, **parameters)
  code = extract_normalized_verilog_code(response.text, bm.name)
  if code == None:
    return "NO_CODE"
  else:
    res = verify.verify_code(bm.specification, code, overwrite_params=bm.generate_params)
    return res.name
    
def run_single_openai(bm, type):
  print("Starting benchmark " + bm.name + "/" + type)
  messages = build_prompt(bm, params=bm.generate_params, template=OpenAIPrompt, mode=type)
  
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.5
  )
  response = completion.choices[0].message.content
  print(response)
  code = extract_normalized_verilog_code(response, bm.name)
  if code == None:
    return "NO_CODE"
  else:
    res = verify.verify_code(bm.specification, code, overwrite_params=bm.generate_params)
    return res.name

async def run_benchmarks(benchmarks, file, example_types = ["self", "bosy", "strix", "none"], run_single=run_single_default):
  # setting up csv writing
  f = open(file, 'w', newline='')
  csvwriter = csv.DictWriter(f, fieldnames=["benchmark"] + example_types, dialect='unix', quoting=csv.QUOTE_NONE)
  csvwriter.writeheader()

  # get event loop to be able to run the requests in parallel
  loop = asyncio.get_event_loop()

  async def _run_single(bm, type):
    try:
      return await loop.run_in_executor(None, run_single, bm, type)
    except TimeoutError:
      return "TIMEOUT"
    except (google.api_core.exceptions.InternalServerError,
          openai.InvalidRequestError):
      return "AI_ERROR" 
    #except Exception as e:
    #  print(e, e.__class__)
  async def run_single_benchmark(bm, i = 0):
    result = {
      "benchmark": bm.name
    }
    print(i)
    await asyncio.sleep(320 * i)  # needed because of quota
    async_res = await asyncio.gather(
      *[_run_single(bm, type) for type in example_types]
    )
    for i, res in enumerate(async_res):
      result[example_types[i]] = res
    csvwriter.writerow(result)

  await asyncio.gather(
    *[run_single_benchmark(bm, i) for i, bm in enumerate(benchmarks)]
  )
  f.close()

await run_benchmarks(benchmarks_auto, run_single=run_single_openai, example_types=["bosy", "strix", "none"],file = "res_openai.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Starting benchmark amba_decomposed_arbiter/bosy
Starting benchmark amba_decomposed_arbiter/strix
Starting benchmark amba_decomposed_arbiter/none
Here is a Verilog module that fulfills the specified expectations:

```verilog
module Module (
  input wire request_2,
  input wire request_1,
  input wire request_0,
  input wire grant_3,
  input wire grant_2,
  input wire grant_1,
  input wire grant_0
);

  reg property1, property2;
  
  always @(posedge clk) begin
    if (reset) begin
      property1 <= 1'b0;
      property2 <= 1'b0;
    end else begin
      property1 <= ((1'b1 && ~request_2 && ~request_1 && ~request_0) -> (1'b1 && ~grant_3 && ~grant_2 && ~grant_1 && ~grant_0)) &&
                   ((1'b1 && ~request_2 && ~request_1 && request_0) -> (1'b1 && ~grant_3 && ~grant_2 && ~grant_1 && grant_0)) &&
                   ((1'b1 && ~request_2 && request_1 && ~request_0) -> (1'b1 && ~grant_3 && ~grant_2 && grant_1 && ~grant_0)) &

RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-7DzntGr1QTXVHmgxAagD0SAi on tokens per min. Limit: 90000 / min. Current: 1371 / min. Contact us through our help center at help.openai.com if you continue to have issues.

Here is a Verilog module that fulfills the specified expectations:

```verilog
module MyModule (
  input wire clk,
  input wire reset,
  input wire allFinished,
  output wire [15:0] finished
);

  reg [15:0] finished_reg;

  always @(posedge clk or posedge reset) begin
    if (reset) begin
      finished_reg <= 16'b0;
    end else if (allFinished) begin
      finished_reg <= {16{1'b1}};
    end else if (finished_reg != 16'b0) begin
      finished_reg <= finished_reg & ~allFinished;
    end
  end

  assign finished = finished_reg;

endmodule
```

This module has three inputs: `clk`, `reset`, and `allFinished`, and one output `finished`, which is a 16-bit vector representing the status of each of the 16 finished signals.

The module uses a register `finished_reg` to store the current state of the finished signals. On each positive edge of the clock (`posedge clk`), the module checks the `reset` signal. If `reset` is high, the `finished_reg` is set to all zeros. If `reset` is low and `all

In [6]:

dot = bosy.synthesize(read_file("detector.tlsf"), target="dot", overwrite_params={"n":2})
print(dot)

digraph graphname {
	_init [style="invis"];
	_init -> s0[label=""];
	s0[shape=rectangle,label="s0"];
	s1[shape=rectangle,label="s1"];
	s1 -> s0 [label="(r_1 ∧ r_0) / g"];
	s1 -> s0 [label="(r_1 ∧ ¬(r_1 ∧ r_0)) / "];
	s1 -> s1 [label="(¬r_1 ∧ ¬(r_1 ∧ r_0)) / "];
	s0 -> s0 [label="(¬(¬r_1 ∧ r_0) ∧ r_0) / g"];
	s0 -> s0 [label="(¬(¬r_1 ∧ r_0) ∧ ¬r_0) / "];
	s0 -> s1 [label="(¬r_1 ∧ r_0) / g"];
}



In [ ]:

contents = read_file("detector.tlsf")
ltl = syfco.convert(contents, "ltl", overwrite_params={"n": 4})
bosy_f = syfco.convert(contents, "bosy", overwrite_params={"n": 4})
bosy_impl = bosy.synthesize(bosy_f)

template = prompting.PromptTemplate()
template.add_example({
    "SPEC": syfco.convert(contents, "ltl", overwrite_params={"n": 2}),
    "IMPL": read_file("../../verilog/detector/detector_2.vl"),
    "PARAMS": "n=2"
})
prompt = template.build_prompt({"SPEC": syfco.convert(contents, "ltl", overwrite_params={"n": 4})})
print(prompt)

You are an expert in writing correct verilog code, that fulfill certain formal properties specified in LTL.
Here is an example for n=2. It satisfies the LTL specification G (F r_0) && G (F r_1) <-> G (F g):
module detector(
  input [1:0] r,
  input clk,
  output reg g
);
  reg [1:0] state;
  initial state = '0;
  always @(posedge clk) begin
    state = state | r;
    g = 0;
    if(state == '1) begin
      g = 1;
      state = '0;
    end
  end
endmodule

Please write a Verilog module fulfilling the following expectations. Make sure the code is fully synthesizable.:
G (F r_0) && G (F r_1) && G (F r_2) && G (F r_3) <-> G (F g)
